<a href="https://colab.research.google.com/github/OGsiji/Transrisk-Innovation/blob/main/Transrisk_Innovation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#from accidents_montreal import get_accident_df
#from utils import init_spark
#from road_network import get_road_df
#from weather import add_weather_columns, extract_year_month_day
from pyspark.sql.functions import row_number, col, rank, avg, split, to_date, min

ModuleNotFoundError: ignored

In [ ]:



import time
import datetime

start = "01/01/" + str('2012')
end = "31/12/" + str('2018')
date = datetime.datetime.strptime(start, "%d/%m/%Y")

dates = list()
while(date != datetime.datetime.strptime(end, "%d/%m/%Y")): 
    date += datetime.timedelta(days=1)
    for i in range(24):
        dates.append((date.strftime("%Y-%m-%d"), i))

In [ ]:
import pandas as pd

In [ ]:
Data = pd.DataFrame(dates, columns=('Date', 'hour'))

In [ ]:
Data.head()

,Date,hour
0,2012-01-02,0
1,2012-01-02,1
2,2012-01-02,2
3,2012-01-02,3
4,2012-01-02,4


In [ ]:
Data.shape

(61344, 2)

In [ ]:
from google.colab import drive
drive.mount("/content/drive/")
%cd "drive/My Drive/"


Mounted at /content/drive/
/content/drive/My Drive


In [ ]:
data2=pd.read_csv('dataBIS correct.csv')

In [ ]:
data2.drop(columns=['DATE','CRASH TIME (HRS)'], inplace=True)

In [ ]:
data2.head()

,DAY,LOCATION,PROBABLE CAUSE(S) OF RTC,ROUTE,VEHICLE CATEGORY,VEHICLE CLASS,Timestamp,month,X,Y
0,SUN,KM8-OOLO,NaN,OGB-OYO,CAR,COMM,05/02/2013 07:00,3,3.89100,7.39975
1,MON,KM8-FLYOVER,NaN,OGB-ILN,BUS,COMM.,4/21/2013 5:00,4,3.90820,7.42622
2,MON,KM24-IROKO,NaN,IBD-OYO,BUS,COMM.,4/27/2013 16:00,4,3.91593,7.51316
3,MON,KM2-SHARPCORNER,NaN,OGB-OYO,TRK,COMM.,06/05/2013 10:00,4,3.91466,7.59099
4,TUE,KM1-AJE IYA,NaN,OGB-OYO,TRL/ MC,COMM/ PVT,06/01/2013 11:00,5,3.92260,7.63765


i

In [ ]:
data2=pd.concat([Data,data2],axis=1)

In [ ]:
data2.head()

,Date,hour,DAY,LOCATION,PROBABLE CAUSE(S) OF RTC,ROUTE,VEHICLE CATEGORY,VEHICLE CLASS,Timestamp,month,X,Y
0,2012-01-02,0,SUN,KM8-OOLO,NaN,OGB-OYO,CAR,COMM,05/02/2013 07:00,3.0,3.89100,7.39975
1,2012-01-02,1,MON,KM8-FLYOVER,NaN,OGB-ILN,BUS,COMM.,4/21/2013 5:00,4.0,3.90820,7.42622
2,2012-01-02,2,MON,KM24-IROKO,NaN,IBD-OYO,BUS,COMM.,4/27/2013 16:00,4.0,3.91593,7.51316
3,2012-01-02,3,MON,KM2-SHARPCORNER,NaN,OGB-OYO,TRK,COMM.,06/05/2013 10:00,4.0,3.91466,7.59099
4,2012-01-02,4,TUE,KM1-AJE IYA,NaN,OGB-OYO,TRL/ MC,COMM/ PVT,06/01/2013 11:00,5.0,3.92260,7.63765


In [ ]:
import random
import numpy as np

#data2["DAY"].fillna(lambda x: random.choice(data2[data2["DAY"] != np.nan]["DAY"]), inplace =True)

In [ ]:
data2["DAY"] = data2["DAY"].apply(lambda x: x.fillna(random.choice(x.dropna())))

AttributeError: ignored

In [ ]:
def na_randomfill(series):
    na_mask = pd.isnull(series)   # boolean mask for null values
    n_null = na_mask.sum()        # number of nulls in the Series
    
    if n_null == 0:
        return series             # if there are no nulls, no need to resample
    
    # Randomly sample the non-null values from our series
    #  only sample this Series as many times as we have nulls 
    fill_values = series[~na_mask].sample(n=n_null, replace=True, random_state=0)

    # This ensures our new values will replace NaNs in the correct locations
    fill_values.index = series.index[na_mask]
    
    return series.fillna(fill_values) 

In [ ]:
data23 = data2.apply(na_randomfill)

In [ ]:
data23['PROBABLE CAUSE(S) OF RTC'].nunique()

103

In [ ]:
data23[data23['LOCATION'] == 'KM7.5-BABA IBADAN VILLAGE']

,Date,hour,DAY,LOCATION,PROBABLE CAUSE(S) OF RTC,ROUTE,VEHICLE CATEGORY,VEHICLE CLASS,Timestamp,month,X,Y
577,2012-01-26,1,TUE,KM7.5-BABA IBADAN VILLAGE,WOV,OYO-OGB,TRK/ CAR,COMM/ PVT,9/26/2018 3:00,8.0,3.91635,7.694
978,2012-02-11,18,TUE,KM7.5-BABA IBADAN VILLAGE,SPV,OYO-OGB,TRK/ CAR,COMM/ PVT,9/26/2018 3:00,8.0,3.91635,7.694
2139,2012-03-31,3,TUE,KM7.5-BABA IBADAN VILLAGE,WOV,OYO-OGB,TRK/ CAR,COMM/ PVT,9/26/2018 3:00,8.0,3.91635,7.694
2869,2012-04-30,13,TUE,KM7.5-BABA IBADAN VILLAGE,WOV,OYO-OGB,TRK/ CAR,COMM/ PVT,9/26/2018 3:00,8.0,3.91635,7.694
3472,2012-05-25,16,TUE,KM7.5-BABA IBADAN VILLAGE,TBT,OYO-OGB,TRK/ CAR,COMM/ PVT,9/26/2018 3:00,8.0,3.91635,7.694
...,...,...,...,...,...,...,...,...,...,...,...,...
59605,2018-10-20,13,TUE,KM7.5-BABA IBADAN VILLAGE,MDV,OYO-OGB,TRK/ CAR,COMM/ PVT,9/26/2018 3:00,8.0,3.91635,7.694
59981,2018-11-05,5,TUE,KM7.5-BABA IBADAN VILLAGE,RTV,OYO-OGB,TRK/ CAR,COMM/ PVT,9/26/2018 3:00,8.0,3.91635,7.694
60070,2018-11-08,22,TUE,KM7.5-BABA IBADAN VILLAGE,SPV,OYO-OGB,TRK/ CAR,COMM/ PVT,9/26/2018 3:00,8.0,3.91635,7.694
60290,2018-11-18,2,TUE,KM7.5-BABA IBADAN VILLAGE,SPV,OYO-OGB,TRK/ CAR,COMM/ PVT,9/26/2018 3:00,8.0,3.91635,7.694


In [ ]:
data23.tail()

,Date,hour,DAY,LOCATION,PROBABLE CAUSE(S) OF RTC,ROUTE,VEHICLE CATEGORY,VEHICLE CLASS,Timestamp,month,X,Y
61339,2018-12-31,19,TUE,KM7.5-BABA IBADAN VILLAGE,"SPV,WOV",OYO-OGB,TRK/ CAR,COMM/ PVT,9/26/2018 3:00,8.0,3.91635,7.69400
61340,2018-12-31,20,MON,KM4.5- CARETAKER,WOV,OGB-OYO,TNK/ MC/ MC,COMM/ COMM/ PVT,12/02/2018 06:00,9.0,3.90930,7.55428
61341,2018-12-31,21,THU,KM6-ARINKINKIN AREA,"DGD,LSV",OGB-OYO,MC/ PICK UP,PVT/ GOVT,12/27/2014 20:00,11.0,4.36135,8.30588
61342,2018-12-31,22,SAT,KM7-DEPEER LIFE AREA,SPV,OYO-IBD,SUV,PVT,06/04/2018 22:00,3.0,4.39034,8.31515
61343,2018-12-31,23,WED,KM1-OYO EAST LG,SPV,OYO-OGB,TRL/ BUS/ MC,COMM/ COMM/ PVT,10/20/2016 5:00,8.0,4.23288,8.12462


In [ ]:
data23.tail()

,Date,hour,DAY,LOCATION,PROBABLE CAUSE(S) OF RTC,ROUTE,VEHICLE CATEGORY,VEHICLE CLASS,Timestamp,month,X,Y,3.0,4.0,5.0,11.0,2.0,8.0,9.0,12.0,1.0,7.0,10.0,6.0
61339,2018-12-31,19,TUE,KM7.5-BABA IBADAN VILLAGE,"SPV,WOV",OYO-OGB,TRK/ CAR,COMM/ PVT,9/26/2018 3:00,8.0,3.91635,7.69400,8.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0
61340,2018-12-31,20,MON,KM4.5- CARETAKER,WOV,OGB-OYO,TNK/ MC/ MC,COMM/ COMM/ PVT,12/02/2018 06:00,9.0,3.90930,7.55428,8.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0
61341,2018-12-31,21,THU,KM6-ARINKINKIN AREA,"DGD,LSV",OGB-OYO,MC/ PICK UP,PVT/ GOVT,12/27/2014 20:00,11.0,4.36135,8.30588,8.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0
61342,2018-12-31,22,SAT,KM7-DEPEER LIFE AREA,SPV,OYO-IBD,SUV,PVT,06/04/2018 22:00,3.0,4.39034,8.31515,8.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0
61343,2018-12-31,23,WED,KM1-OYO EAST LG,SPV,OYO-OGB,TRL/ BUS/ MC,COMM/ COMM/ PVT,10/20/2016 5:00,8.0,4.23288,8.12462,8.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0


In [ ]:
import numpy as np
def fill_with_random(data2, DAY):
    '''Fill `df2`'s column with name `column` with random data based on non-NaN data from `column`'''
    data2 = data2.copy()
    data2['DAY'] = data2['DAY'].apply(lambda x: np.random.choice(data2['DAY'].dropna().values) if np.isnan(x) else x)
    return data2

In [ ]:
len(Data)

61344